In [1]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd
import sqlite3


In [2]:
#coneect to database

In [3]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [4]:
cur = conn.cursor()

In [5]:
query1 = """
    SELECT *
    FROM Bookings
    """

In [6]:
cur.execute(query1)

In [7]:
# better, more convenience 
Bookings = pd.read_sql_query("SELECT * FROM Bookings", conn)
Facilities = pd.read_sql_query("SELECT * FROM Facilities", conn)
Members = pd.read_sql_query("SELECT * FROM Members", conn)

In [8]:
Facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [9]:
Members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [10]:
Bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


## Q10:

In [11]:
Earning = {}
Revenue = {}
for i in range(len(Facilities)):
    Earning[i]=0
    Revenue[i]=0
print(Earning)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}


In [12]:

for i in range(len(Bookings)):
    for j in range(len(Facilities)):
        if Bookings.loc[i,'facid'] ==j:
            if Bookings.loc[i,'memid'] == 0:
                Earning[j] += Facilities.loc[j,'guestcost']
                
            else: 
                Earning[j] += Facilities.loc[j,'membercost']

for j in range(len(Facilities)):
    Revenue[j] =Earning[j] - Facilities.loc[j,'initialoutlay'] - Facilities.loc[j,'monthlymaintenance']*3.0
            
print(Revenue)

{0: -6560.0, 1: -4395.0, 2: -3545.5, 3: -260.0, 4: 7807.900000000045, 5: -6012.700000000003, 6: -270.0, 7: -380.0, 8: -180.0}


In [13]:
df_earning = pd.DataFrame.from_dict([Earning,Revenue]).transpose()
df_earning.columns = ['earning','Revenue']
df_earning.head(20)

,earning,Revenue
0,4040.0,-6560.0
1,4205.0,-4395.0
2,604.5,-3545.5
3,90.0,-260.0
4,20807.9,7807.9
5,6987.3,-6012.7
6,4970.0,-270.0
7,115.0,-380.0
8,265.0,-180.0


In [14]:
#Q10 answer:
df_earning[df_earning['Revenue']<1000]

,earning,Revenue
0,4040.0,-6560.0
1,4205.0,-4395.0
2,604.5,-3545.5
3,90.0,-260.0
5,6987.3,-6012.7
6,4970.0,-270.0
7,115.0,-380.0
8,265.0,-180.0


## Q11

In [15]:
query11 = """
    SELECT memid, surname,firstname, recommendedby
    FROM Members
    WHERE surname NOT LIKE 'GUEST%'
    """

In [16]:
Members_recommend = pd.read_sql_query(query11, conn)

In [17]:
Members_recommend.head(10)

,memid,surname,firstname,recommendedby
0,1,Smith,Darren,
1,2,Smith,Tracy,
2,3,Rownam,Tim,
3,4,Joplette,Janice,1
4,5,Butters,Gerald,1
5,6,Tracy,Burton,
6,7,Dare,Nancy,4
7,8,Boothe,Tim,3
8,9,Stibbons,Ponder,6
9,10,Owen,Charles,1


In [18]:
df = Members_recommend
rec_name = []
for i in range(len(df)-1):
    if df.loc[i,'recommendedby'] is not '':
        number = df.loc[i,'recommendedby']
        rec_name.append(
    {
        'rec_sur': Members_recommend[Members_recommend['memid'] == int(number)]['surname'].values[0],
        'rec_first': Members_recommend[Members_recommend['memid'] == int(number)]['firstname'].values[0],
    }
    )
    else:
        rec_name.append(
    {
        'rec_sur':'', 
        'rec_first':'', 
    }
    )

df_recname = pd.DataFrame(rec_name)
df_con = pd.concat([Members_recommend,df_recname],axis = 1)
df_con = df_con.sort_values(['rec_sur','rec_first'])
df_con.head(10)

,memid,surname,firstname,recommendedby,rec_sur,rec_first
0,1,Smith,Darren,,,
1,2,Smith,Tracy,,,
2,3,Rownam,Tim,,,
5,6,Tracy,Burton,,,
12,13,Farrell,Jemima,,,
23,28,Farrell,David,,,
26,33,Tupperware,Hyacinth,,,
20,24,Sarwin,Ramnaresh,15,Bader,Florence
19,22,Coplin,Joan,16,Baker,Timothy
17,20,Genting,Matthew,5,Butters,Gerald


## Q12 Find the facilities with their usage by member, but not guests

In [19]:
query12 = """
    SELECT COUNT(memid), facid
    FROM Bookings
    WHERE memid != 0
    GROUP BY facid
    """

In [20]:
Members_usage = pd.read_sql_query(query12, conn)

In [21]:
Members_usage.head(10)

,COUNT(memid),facid
0,308,0
1,276,1
2,344,2
3,385,3
4,421,4
5,27,5
6,195,6
7,421,7
8,783,8


In [22]:
#python coding method to verify

In [23]:
Members_count = pd.read_sql_query("SELECT memid, facid FROM Bookings WHERE memid != 0", conn)
Members_count.head()

,memid,facid
0,1,3
1,1,4
2,1,7
3,1,8
4,1,8


In [24]:
count = {}
for i in range(len(Facilities)):
     count[i]=0
for i in range(len(Members_count)):
    for j in range(len(Facilities)):
        if Members_count.loc[i,'facid'] ==j:
            count[j] +=1
            
print(count)

{0: 308, 1: 276, 2: 344, 3: 385, 4: 421, 5: 27, 6: 195, 7: 421, 8: 783}


## Q13 Find the facilities usage by month, but not guests 

In [25]:
query13 = """
    SELECT facid,
        COUNT(CASE WHEN starttime Like '2012-07%' AND memid != 0 THEN memid END)AS Julymemid, 
        COUNT(CASE WHEN starttime Like '2012-08%' AND memid != 0 THEN memid END)AS Augmemid,
        COUNT(CASE WHEN starttime Like '2012-09%' AND memid != 0 THEN memid END)AS Sepmemid
    FROM Bookings
    GROUP BY facid
    """

In [26]:
Use_count = pd.read_sql_query(query13, conn)
Use_count

,facid,Julymemid,Augmemid,Sepmemid
0,0,65,111,132
1,1,41,109,126
2,2,51,132,161
3,3,48,143,194
4,4,77,153,191
5,5,4,9,14
6,6,23,85,87
7,7,68,154,199
8,8,103,272,408
